In [1]:
cd '/kaggle/working'

In [2]:
import os
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [3]:
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-deprecated=legacy-resolver .

In [4]:
!pip uninstall tensorflow-gpu -y
!pip uninstall tensorflow -y
!pip install tensorflow-gpu==2.4.1

In [5]:
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [6]:
pwd

In [7]:
!python ./object_detection/builders/model_builder_tf2_test.py

In [8]:
!pip install untangle

In [10]:
cd '../'

In [11]:
len(os.listdir('../input/bccd-dataset/BCCD/JPEGImages'))

In [12]:
import PIL
import tensorflow as tf
import hashlib
import io
import os
import untangle
from object_detection.utils import dataset_util

def xml_to_tf_example(xml_obj):
    label_map_dict = {'WBC':1 , 'RBC':2 , 'Platelets':3}
    filename = xml_obj.annotation.filename.cdata
    full_path = '../input/bccd-dataset/BCCD/JPEGImages/' + filename
    with tf.io.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    width = int(xml_obj.annotation.size.width.cdata)
    height = int(xml_obj.annotation.size.height.cdata)

    xmin = []
    ymin = []
    xmax = []
    ymax = []

    classes = []
    classes_text = []
    truncated = []

    for obj in xml_obj.annotation.object:

        xmin.append(float(obj.bndbox.xmin.cdata) / width)
        ymin.append(float(obj.bndbox.ymin.cdata) / height)
        xmax.append(float(obj.bndbox.xmax.cdata) / width)
        ymax.append(float(obj.bndbox.ymax.cdata) / height)
        classes_text.append(obj.name.cdata.encode('utf8'))
        classes.append(label_map_dict[obj.name.cdata])
        truncated.append(int(obj.truncated.cdata))
    #print(width)
    #print(xmin)
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/truncated': dataset_util.int64_list_feature(truncated),
    }))
    return example

data_dir = '../input/bccd-dataset/BCCD'

tfrecord_path = './train.tfrecord'
i = 0
writer = tf.io.TFRecordWriter(tfrecord_path)

annotations_dir = os.path.join(data_dir, 'Annotations')
examples_list = os.listdir(annotations_dir)
for idx, example in enumerate(examples_list):
    if(i>300):
        break
    if example.endswith('.xml'):
        path = os.path.join(annotations_dir, example)
        xml_obj = untangle.parse(path)
        tf_example = xml_to_tf_example(xml_obj)
        writer.write(tf_example.SerializeToString())
    i+=1
        

writer.close()

In [13]:
%%writefile ./label_map.pbtxt
item {
  id: 1
  name: 'RBC'
}
item {
  id: 2
  name: 'WBC'
}
item {
  id: 3
  name: 'Platelets'
}

In [14]:
MODELS_CONFIG = {
        'faster_rcnn_resnet101': {
        'model_name': 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8',
        'base_pipeline_file': 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.config',
        'pretrained_checkpoint': 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz',
        'batch_size': 4
    },

}

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'faster_rcnn_resnet101'

num_steps = 3000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [15]:
%mkdir ./models/research/deploy/
%cd ./models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

In [16]:
%cd /kaggle/working/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [17]:
pipeline_fname = '/kaggle/working/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/kaggle/working/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'
num_classes = 3

In [20]:
cd '../'

In [21]:
import re

#%cd /kaggle/working/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format('./train.tfrecord'), s)
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format('./test.tfrecord'), s)

    # label_map_path
    s = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format('./label_map.pbtxt'), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)


In [22]:
%cat pipeline_file.config

In [23]:
pipeline_file = '/kaggle/working/pipeline_file.config'
model_dir = '/kaggle/working/training/'

In [24]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [25]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [26]:
%ls './training/'

In [27]:
import pathlib

filenames = list(pathlib.Path('./training/').glob('*.index'))

filenames.sort()
print(filenames)

#recover our saved model
pipeline_config = pipeline_file
#generally you want to put the last ckpt from training in here
model_dir = str(filenames[-1]).replace('.index','')
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(str(filenames[-1]).replace('.index','')))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [28]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
import random
import matplotlib.patches as patches
data_dir = '../input/bccd-dataset/BCCD'
annotations_dir = os.path.join(data_dir, 'Annotations')
examples_list = os.listdir(annotations_dir)
path = os.path.join(annotations_dir, 'BloodImage_00301.xml')
#print(path)
xml_obj = untangle.parse(path)
lst1 = []
for obj in xml_obj.annotation.object:
    lst2 = []
    lst2.append(float(obj.bndbox.xmin.cdata))
    lst2.append(float(obj.bndbox.ymin.cdata))
    lst2.append(float(obj.bndbox.xmax.cdata))
    lst2.append(float(obj.bndbox.ymax.cdata))
    lst1.append(lst2)

plt.figure(figsize=(6,6))
plt.imshow(load_image_into_numpy_array('../input/bccd-dataset/BCCD/JPEGImages/BloodImage_00301.jpg'))
ax = plt.gca()
#fig, ax = plt.subplots()
#ax.imshow(image_np_with_detections)
for i in lst1:
    rect = patches.Rectangle((i[0],i[1]), (i[2]-i[0]) , (i[3]-i[1]), linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.savefig('Original_301_SSD_ResNet101_V1_FPN_640x640.jpeg')
plt.show()
#plt.savefig('Original_360_SSD_ResNet50_V1_FPN_640x640.jpeg')

In [29]:
def draw_bbox(image, box, label, color):   
    alpha = 0.1
    alpha_box = 0.4
    overlay_bbox = image.copy()
    overlay_text = image.copy()
    output = image.copy()
    output = cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),color, 2)
    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)[0]
    cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, (box[0], box[1]-7-text_height),(box[0]+text_width+2, box[1]), (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.putText(output, label.upper(), (box[0], box[1]-5),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    return output

In [41]:
pwd

In [31]:
import random
import matplotlib.patches as patches
import cv2 

label2color = [[59, 238, 119], [222, 21, 229], [94, 49, 164], [206, 221, 133], [117, 75, 3],
                 [210, 224, 119], [211, 176, 166], [63, 7, 197], [102, 65, 77], [194, 134, 175],
                 [209, 219, 50], [255, 44, 47], [89, 125, 149], [110, 27, 100]]

TEST_IMAGE_PATHS = os.listdir('../input/bccd-dataset/BCCD/JPEGImages')
image_path = '../input/bccd-dataset/BCCD/JPEGImages/BloodImage_00360.jpg'
#print(image_path)
image_np = load_image_into_numpy_array(image_path)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)
#print(detections)
#print(predictions_dict)
label_id_offset = 1
image_np_with_detections = image_np.copy()
min_score_thresh = 0.2
width, height, _ = image_np.shape
boxes = detections['detection_boxes'][0].numpy()
boxes = boxes[:,[1, 0, 3, 2]]*np.array([height, width, height, width])

for box, label_id, score in zip(boxes,(detections['detection_classes'][0].numpy() + label_id_offset).astype(int),detections['detection_scores'][0].numpy()):
    if (score > min_score_thresh):
        image_np_with_detections = draw_bbox(image_np_with_detections, list(np.int_(box)),category_index[label_id]['name']+'_'+str(round(score*100,1))+'%',label2color[label_id-1])

#viz_utils.visualize_boxes_and_labels_on_image_array(
#      image_np_with_detections,
#      detections['detection_boxes'][0].numpy(),
#      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
#      detections['detection_scores'][0].numpy(),
#      category_index,
#      use_normalized_coordinates=True,
#      max_boxes_to_draw=200,
#      min_score_thresh=.5,
#      agnostic_mode=False,
#)
plt.figure(figsize=(6,6))
plt.imshow(image_np_with_detections)
plt.savefig('Predicted_360_Faster_RCNN_ResNet101_V1_FPN_1024x1024.jpeg')
plt.show()
#plt.savefig('Predicted_360_SSD_ResNet50_V1_FPN_640x640.jpeg')

#ax = plt.gca()
#fig, ax = plt.subplots()
#ax.imshow(image_np_with_detections)
#for i in detections['detection_boxes'][0].numpy():
#    rect = patches.Rectangle((i[0]*512,i[1]*512), (i[2]-i[0])*512 , (i[3]-i[1])*512, linewidth=1, edgecolor='r', facecolor='none')
#    ax.add_patch(rect)
#plt.show()

In [32]:
pred = []
test_images_list = os.listdir('../input/bccd-dataset/BCCD/JPEGImages')
print(len(test_images_list))
#print(test_images_list)
# Doing though each image
j = 0
for file_path in (test_images_list):
    print(j)
    if(j<300):
        j+=1
        continue
    img = cv2.imread('../input/bccd-dataset/BCCD/JPEGImages/' + file_path)
    
    width,height,_ = img.shape
    image_np = load_image_into_numpy_array('../input/bccd-dataset/BCCD/JPEGImages/' + file_path)
    input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
    outputs, predictions_dict, shapes = detect_fn(input_tensor)
    #print(outputs['detection_boxes'][0][0])
    #print(outputs['detection_classes'][0][0])
    #print(outputs['detection_scores'][0][0])
    #break
    #outputs = predictor(img)
    image_path, image_file_name = os.path.split(file_path)
    #image_id = int(image_file_name.split(".")[0])+1
    i = 0
    for boxes in outputs['detection_boxes'][0]:
        if(outputs['detection_scores'][0][i].numpy().tolist() > 0.35):
            # Converting thr bounding boxes from (x1, y1, x2, y2) to (x, y, w, h)
            boxes = boxes.numpy().tolist()
            preprocessed_box = [boxes[0]*height, boxes[1]*width, abs(boxes[0] - boxes[2])*height, abs(boxes[1] - boxes[3])*width]
            pred.append({
                "image_id": file_path,
                "category_id": outputs['detection_classes'][0][i].numpy().tolist(),
                "bbox": preprocessed_box,
                "score":outputs['detection_scores'][0][i].numpy().tolist()
            })
        i+=1
    j+=1

In [34]:
#pred

In [35]:
import json
with open('predictions_faster_rcnn_resnet101_1024X1024.json', 'w') as f:
    json.dump(pred, f)